<a href="https://colab.research.google.com/github/Shashank-Ojha/WordFormer/blob/main/trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install torch
%load_ext autoreload
%autoreload 2

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from utils.attention_block import SelfAttention
from utils.prepare_data import read_file
from utils.prepare_data import get_vocab
from utils.prepare_data import train_validation_split
from utils.prepare_data import get_batch

# Set a seed for reproducibility
torch.manual_seed(0)

In [8]:
# Read text file, convert each token (char) into integers, and split data into train and validation data.
text = read_file('shakespeare.txt')

# Here are all the unique characters that occur in this text
vocab, vocab_size = get_vocab(text)

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(vocab) }
itos = { i:ch for i,ch in enumerate(vocab) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
train_data, val_data = train_validation_split(data, 0.9)

In [15]:
print(data.shape)

torch.Size([1115394])


In [38]:
# ------ Hyperparameters ------

# -- Data Breakdown
batch_size = 4
block_size = 8

# -- Network Parameters
# Embedding Block
num_embeddings = vocab_size
embedding_dim = 8

# Attention Block
kq_dim = 7
v_dim = 6

# -- Training Parameters

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [51]:
# Both x and y are (batch_size, block_size)
x, y = get_batch(train_data, batch_size, 2)

In [67]:
# Define Network

# Input Shape (batch_size, block_size). 
# Note block_size is not a parameters to the model, so it can accept arbitary
# length contexts.
model = nn.Sequential(
    # Transforms shape to (batch_size, block_size, embedding_dim)
    nn.Embedding(num_embeddings, embedding_dim),
    # Transforms shape to (batch_size, block_size, v_dim)
    SelfAttention(embedding_dim, kq_dim, v_dim),
    # Transforms shape to (batch_size, block_size, vocab_size)
    nn.Linear(v_dim, vocab_size)
    
)

print(model(x).shape)

def generate(model, context, max_context_size, max_new_tokens):
    """
    context: array tokens with shape (B, t), where t can be any length.
    """
    for _ in range(max_new_tokens):
        sub_context = context if context.shape[1] < max_context_size else context[:, -max_context_size:]
        # Shape = (batch_size, block_size, vocab_size)
        logits = model(sub_context)
        # Get the last character's predictions. Shape = (B, vocab_size)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1) 
        
        # Sample from the distribution. Shape = (B, 1)
        preds = torch.multinomial(probs, num_samples=1)   
        
        # Add to context
        context = torch.cat((context, preds), dim=1)
    return context
        
        

context = torch.zeros((1, 1), dtype=torch.long, device=device)
new_text = generate(model, context, block_size, max_new_tokens=500)
print(decode(new_text[0].tolist()))

torch.Size([4, 2, 65])

rY,H:FLJ g3Si 
Q,OBupEko;.qac&$YIIDF'3.XgpfpLrKq&P'XsEsagmHzzvgMcYN?afrfoN?SljfGJdC?LPaROczL
'3Z:D!cL.gOx,kUBVSvjY'BPLjV&rXXbG!zjzgl-KsOvAfqt!ICB3ds3ox&YzaXODpqUi$p33ilm$RlWjab3OA,lX'fc scRXjZVvpuFpwmqof,YtETzUvi3lnM:i,pE?aPCeSjLw3J-j$qaiCAhhlb3 ukqVcA3RHHElR3$twE$dMh 3lS
DFpH:lyK.m,trFVjSCZfBy:.IEEYfk'EFH;wfZytP&
T.PjtrcNIft?3$s-LlcmEytKzdDX!f:mVavEaL,MCy
3&iF:t
vVWdLxJJBCmggkJM$3ciyLWVJNY;D,Laasp?eJKgMigeL.riOmnluucfstUIONN,ccBJHJSedTHGX&Jiz'bsA3uJDFX;to?Pt aHJPlHvJlK-cjB ,Tm$c$?NE;:FlcBnsFr!o
